# 0. Premilinary Works
How big is the file, number of rows, etc.

In [1]:
cd ../data

/home/angelrps/git/MasterDataScience_FinalProject/data


In [2]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 3660944
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rw-rw-rw- 1 angelrps angelrps   13400119 Jun 23 09:34 Data_Cleaned_To_Model.csv
-rw-rw-rw- 1 angelrps angelrps   24377425 Jun 20 06:26 Data_Taxis_2017_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26220204 Jun 20 04:20 Data_Taxis_2018_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26759445 Jun 20 00:08 Data_Taxis_2019_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps    3755058 Jun 22 10:01 NY_taxi_zones.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionar

In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
#!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [5]:
# Count Number of Lines
!bzcat -c ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
!bzcat -c ./2018_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
!bzcat -c ./2017_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l

# 2017_...113.496.875 lines
# 2018_...112.234.627 lines
# 2019_...84.399.020 lines
# The process took about 15 min to calculate the lines

84399020
112234627
113496875


#### I will start exploring the 2019 dataset
I will try to use bash to clean the data as much as I can before importing a Data Frame because I believe is faster.
From a previous exploration I concluded:
1. There are trips with 0 'trip_distance'. I will remove all trips with 'trip_distance' < 0.06 (100 meters) because I consider them either measurement errors or non representative data.
2. There are trips with either negative or very little 'fare_amount' (~0.01). I will remove rows with fare amount<1$ because I consider them measurement errors.

In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


### 1. Clean up dataset taking just a sample

The following cleaning actions are based on the dataset exploration made in the notebok 'Data_Analysis.ipynb'

In [36]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### 1. Import part of the csv so the memory can handle it

In [35]:
# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

(9999, 4)

### 2. Let's remove trips under 0.06 (100m). I consider them either errors or not relevant

In [34]:
# RECAP 2

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all trips with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
dfsample[dfsample['trip_distance']<0.06].index # this should be none

Int64Index([], dtype='int64')

### 3.  Now let's remove negative 'fare_amount' or <1$. I consider them errors or not relevant.

In [28]:
# RECAP 3

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)
dfsample[dfsample['fare_amount']<1] # this should be none

,pickup_datetime,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge


### 4. Once we have removed unwanted rows, let´s select the two columns of interest:
- 'pickup_datetime'
- 'PULocationID'

In [24]:
# RECAP 4

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
dfsample.head()

,pickup_datetime,PULocationID
1,2019-04-29 10:52:08,186
2,2019-04-29 11:28:40,238
3,2019-04-29 11:28:06,138
4,2019-04-29 11:39:32,138
5,2019-04-29 11:06:45,141


In [26]:
year = 2019
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(date(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(date(year+1,1,1))].index, inplace=True)

### 5. Drop unwanted years
Eventhough the data states the year, I have detected row with wrong years so I´ll filter them out.

In [70]:
# RECAP 5

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2019
dfsample = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
#dfsample.head()

# 5. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(date(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(date(year+1,1,1))].index, inplace=True)

### 6. Groupby Hour and LocationID

In [117]:
# RECAP 6

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2019
dfsample = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
#dfsample.head()

# 5. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(date(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(date(year+1,1,1))].index, inplace=True)

# 6. Groupby HOUR and LocationID
    # set time period index
dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    # Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    # Create new column to count Number of Pickups per LocationID
dfsample['NoOfPickups'] =1
    # Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()

dfsample.head(50)

,pickup_datetime,PULocationID,NoOfPickups
0,2019-04-29 05:00,90,1
1,2019-04-29 06:00,68,1
2,2019-04-29 06:00,88,1
3,2019-04-29 06:00,107,1
4,2019-04-29 06:00,113,1
5,2019-04-29 06:00,125,1
6,2019-04-29 06:00,132,1
7,2019-04-29 06:00,138,1
8,2019-04-29 06:00,236,1
9,2019-04-29 06:00,237,1


### 7. Break down datetime in other useful variables
    - year
    - month
    - day
    - week
    - dayofweek
    - isweekend

In [188]:
# RECAP 7

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2019
dfsample = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
#dfsample.head()

# 5. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(date(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(date(year+1,1,1))].index, inplace=True)

# 6. Groupby HOUR and LocationID
    # set time period index
dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    # Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    # Create new column to count Number of Pickups per LocationID
dfsample['NoOfPickups'] =1
    # Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()

# 7. Create new variables from datetime
dfsample['year'] = dfsample['pickup_datetime'].dt.year
dfsample['month'] = dfsample['pickup_datetime'].dt.month
dfsample['day'] = dfsample['pickup_datetime'].dt.day
dfsample['hour'] = dfsample['pickup_datetime'].dt.hour
dfsample['week'] = dfsample['pickup_datetime'].dt.week
dfsample['dayofweek'] = dfsample['pickup_datetime'].dt.dayofweek
    # 6.1 create variable 'isweekend'
mask = (dfsample['dayofweek'] == 5) | (dfsample['dayofweek'] == 6)
dfsample['isweekend'] = np.where(mask, 1, 0)

dfsample.sample(50)

,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend
16,2019-04-29 08:00,237,1,2019,4,29,8,18,0,0
28,2019-04-29 10:00,107,4,2019,4,29,10,18,0,0
192,2019-04-29 12:00,79,73,2019,4,29,12,18,0,0
12,2019-04-29 07:00,162,1,2019,4,29,7,18,0,0
150,2019-04-29 11:00,237,300,2019,4,29,11,18,0,0
220,2019-04-29 12:00,146,6,2019,4,29,12,18,0,0
191,2019-04-29 12:00,76,1,2019,4,29,12,18,0,0
275,2019-04-29 13:00,230,1,2019,4,29,13,18,0,0
203,2019-04-29 12:00,113,64,2019,4,29,12,18,0,0
11,2019-04-29 07:00,113,1,2019,4,29,7,18,0,0


In [197]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dr = pd.date_range(start='2019-01-01', end='2019-12-31')
data = pd.date_range('26/5/2019', periods = 5, freq ='D')
df = pd.DataFrame()
df2 = pd.DataFrame()
df['Date'] = dfsample['pickup_datetime']
df2['Date'] = data
df = df.append(df2)
cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

df['Holiday'] = df['Date'].isin(holidays)
print (df.head())
print(df.info())

               Date  Holiday
0  2019-04-29 05:00    False
1  2019-04-29 06:00    False
2  2019-04-29 06:00    False
3  2019-04-29 06:00    False
4  2019-04-29 06:00    False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 4
Data columns (total 2 columns):
Date       284 non-null object
Holiday    284 non-null bool
dtypes: bool(1), object(1)
memory usage: 4.7+ KB
None


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
Date       31 non-null datetime64[ns]
Holiday    31 non-null bool
dtypes: bool(1), datetime64[ns](1)
memory usage: 407.0 bytes


### 2. Test Big Data approach on a sample

In [7]:
# LIBRARIES
import pandas as pd
pd.set_option('display.max_columns', None)

In [41]:
# CODE TO RUN

DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

dfsample = dfsample[['pickup_datetime','PULocationID']]

dfsample.set_index('pickup_datetime', inplace=True)

dfsample = dfsample.to_period("H")

dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

dfsample['NoOfPickups'] =1

dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()

In [10]:
# ITERATION CODE WITH CHUNKS

# 1. Get 10000 rows sample iterator with 3000 chunks
dfsi = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))    
    
    # CODE TO RUN
    DistIndex_to_drop = dfs[dfs['trip_distance'] < 0.06].index
    dfs.drop(DistIndex_to_drop, inplace=True)
    
    FareIndex_to_drop = dfs[dfs['fare_amount']<1].index
    dfs.drop(FareIndex_to_drop, inplace=True)

    dfs = dfs[['pickup_datetime','PULocationID']]

    dfs.set_index('pickup_datetime', inplace=True)

    dfs = dfs.to_period("H")

    dfs.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

    dfs['NoOfPickups'] =1
    
    result=dfs.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
result_all.head()

0 3000
1 3000
2 3000
3 999


I check that the 'result_all' dataframe and 'dfsample' are the same:
- Shape is  in both (279,3)
- Head and Tail rows are the same
- Taking a random row (e.g. row 52), they are both the same.

In [84]:
print(dfsample.tail())
print(result_all.tail())

      pickup_datetime  PULocationID  NoOfPickups
274  2019-04-29 13:00           229            1
275  2019-04-29 13:00           230            1
276  2019-04-29 13:00           231            1
277  2019-04-29 13:00           234            1
278  2019-04-29 13:00           236            2
      pickup_datetime  PULocationID  NoOfPickups
274  2019-04-29 13:00           229            1
275  2019-04-29 13:00           230            1
276  2019-04-29 13:00           231            1
277  2019-04-29 13:00           234            1
278  2019-04-29 13:00           236            2


In [85]:
print(dfsample.head())
print(result_all.head())

    pickup_datetime  PULocationID  NoOfPickups
0  2019-04-29 05:00            90            1
1  2019-04-29 06:00            68            1
2  2019-04-29 06:00            88            1
3  2019-04-29 06:00           107            1
4  2019-04-29 06:00           113            1
    pickup_datetime  PULocationID  NoOfPickups
0  2019-04-29 05:00            90            1
1  2019-04-29 06:00            68            1
2  2019-04-29 06:00            88            1
3  2019-04-29 06:00           107            1
4  2019-04-29 06:00           113            1


In [86]:
print(dfsample.loc[52])
print(result_all.loc[52])

pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
Name: 52, dtype: object
pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
Name: 52, dtype: object


### 3. Run program with Big Data

In [12]:

# 1. Get whole file with 3000 chunks
dfsi = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=10000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))    
    
    # CODE TO RUN
    DistIndex_to_drop = dfs[dfs['trip_distance'] < 0.06].index
    dfs.drop(DistIndex_to_drop, inplace=True)
    
    FareIndex_to_drop = dfs[dfs['fare_amount']<1].index
    dfs.drop(FareIndex_to_drop, inplace=True)

    dfs = dfs[['pickup_datetime','PULocationID']]

    dfs.set_index('pickup_datetime', inplace=True)

    dfs = dfs.to_period("H")

    dfs.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

    dfs['NoOfPickups'] =1
    
    result=dfs.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
result_all.head()

0 10000
1 10000
2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 10000
10 10000
11 10000
12 10000
13 10000
14 10000
15 10000
16 10000
17 10000
18 10000
19 10000
20 10000
21 10000
22 10000
23 10000
24 10000
25 10000
26 10000
27 10000
28 10000
29 10000
30 10000
31 10000
32 10000
33 10000
34 10000
35 10000
36 10000
37 10000
38 10000
39 10000
40 10000
41 10000
42 10000
43 10000
44 10000
45 10000
46 10000
47 10000
48 10000
49 10000
50 10000
51 10000
52 10000
53 10000
54 10000
55 10000
56 10000
57 10000
58 10000
59 10000
60 10000
61 10000
62 10000
63 10000
64 10000
65 10000
66 10000
67 10000
68 10000
69 10000
70 10000
71 10000
72 10000
73 10000
74 10000
75 10000
76 10000
77 10000
78 10000
79 10000
80 10000
81 10000
82 10000
83 10000
84 10000
85 10000
86 10000
87 10000
88 10000
89 10000
90 10000
91 10000
92 10000
93 10000
94 10000
95 10000
96 10000
97 10000
98 10000
99 10000
100 10000
101 10000
102 10000
103 10000
104 10000
105 10000
106 10000
107 10000
108 10000
109 10000
110 10000


831 10000
832 10000
833 10000
834 10000
835 10000
836 10000
837 10000
838 10000
839 10000
840 10000
841 10000
842 10000
843 10000
844 10000
845 10000
846 10000
847 10000
848 10000
849 10000
850 10000
851 10000
852 10000
853 10000
854 10000
855 10000
856 10000
857 10000
858 10000
859 10000
860 10000
861 10000
862 10000
863 10000
864 10000
865 10000
866 10000
867 10000
868 10000
869 10000
870 10000
871 10000
872 10000
873 10000
874 10000
875 10000
876 10000
877 10000
878 10000
879 10000
880 10000
881 10000
882 10000
883 10000
884 10000
885 10000
886 10000
887 10000
888 10000
889 10000
890 10000
891 10000
892 10000
893 10000
894 10000
895 10000
896 10000
897 10000
898 10000
899 10000
900 10000
901 10000
902 10000
903 10000
904 10000
905 10000
906 10000
907 10000
908 10000
909 10000
910 10000
911 10000
912 10000
913 10000
914 10000
915 10000
916 10000
917 10000
918 10000
919 10000
920 10000
921 10000
922 10000
923 10000
924 10000
925 10000
926 10000
927 10000
928 10000
929 10000
930 10000


1592 10000
1593 10000
1594 10000
1595 10000
1596 10000
1597 10000
1598 10000
1599 10000
1600 10000
1601 10000
1602 10000
1603 10000
1604 10000
1605 10000
1606 10000
1607 10000
1608 10000
1609 10000
1610 10000
1611 10000
1612 10000
1613 10000
1614 10000
1615 10000
1616 10000
1617 10000
1618 10000
1619 10000
1620 10000
1621 10000
1622 10000
1623 10000
1624 10000
1625 10000
1626 10000
1627 10000
1628 10000
1629 10000
1630 10000
1631 10000
1632 10000
1633 10000
1634 10000
1635 10000
1636 10000
1637 10000
1638 10000
1639 10000
1640 10000
1641 10000
1642 10000
1643 10000
1644 10000
1645 10000
1646 10000
1647 10000
1648 10000
1649 10000
1650 10000
1651 10000
1652 10000
1653 10000
1654 10000
1655 10000
1656 10000
1657 10000
1658 10000
1659 10000
1660 10000
1661 10000
1662 10000
1663 10000
1664 10000
1665 10000
1666 10000
1667 10000
1668 10000
1669 10000
1670 10000
1671 10000
1672 10000
1673 10000
1674 10000
1675 10000
1676 10000
1677 10000
1678 10000
1679 10000
1680 10000
1681 10000
1682 10000

2337 10000
2338 10000
2339 10000
2340 10000
2341 10000
2342 10000
2343 10000
2344 10000
2345 10000
2346 10000
2347 10000
2348 10000
2349 10000
2350 10000
2351 10000
2352 10000
2353 10000
2354 10000
2355 10000
2356 10000
2357 10000
2358 10000
2359 10000
2360 10000
2361 10000
2362 10000
2363 10000
2364 10000
2365 10000
2366 10000
2367 10000
2368 10000
2369 10000
2370 10000
2371 10000
2372 10000
2373 10000
2374 10000
2375 10000
2376 10000
2377 10000
2378 10000
2379 10000
2380 10000
2381 10000
2382 10000
2383 10000
2384 10000
2385 10000
2386 10000
2387 10000
2388 10000
2389 10000
2390 10000
2391 10000
2392 10000
2393 10000
2394 10000
2395 10000
2396 10000
2397 10000
2398 10000
2399 10000
2400 10000
2401 10000
2402 10000
2403 10000
2404 10000
2405 10000
2406 10000
2407 10000
2408 10000
2409 10000
2410 10000
2411 10000
2412 10000
2413 10000
2414 10000
2415 10000
2416 10000
2417 10000
2418 10000
2419 10000
2420 10000
2421 10000
2422 10000
2423 10000
2424 10000
2425 10000
2426 10000
2427 10000

3082 10000
3083 10000
3084 10000
3085 10000
3086 10000
3087 10000
3088 10000
3089 10000
3090 10000
3091 10000
3092 10000
3093 10000
3094 10000
3095 10000
3096 10000
3097 10000
3098 10000
3099 10000
3100 10000
3101 10000
3102 10000
3103 10000
3104 10000
3105 10000
3106 10000
3107 10000
3108 10000
3109 10000
3110 10000
3111 10000
3112 10000
3113 10000
3114 10000
3115 10000
3116 10000
3117 10000
3118 10000
3119 10000
3120 10000
3121 10000
3122 10000
3123 10000
3124 10000
3125 10000
3126 10000
3127 10000
3128 10000
3129 10000
3130 10000
3131 10000
3132 10000
3133 10000
3134 10000
3135 10000
3136 10000
3137 10000
3138 10000
3139 10000
3140 10000
3141 10000
3142 10000
3143 10000
3144 10000
3145 10000
3146 10000
3147 10000
3148 10000
3149 10000
3150 10000
3151 10000
3152 10000
3153 10000
3154 10000
3155 10000
3156 10000
3157 10000
3158 10000
3159 10000
3160 10000
3161 10000
3162 10000
3163 10000
3164 10000
3165 10000
3166 10000
3167 10000
3168 10000
3169 10000
3170 10000
3171 10000
3172 10000

...

6062 10000
6063 10000
6064 10000
6065 10000
6066 10000
6067 10000
6068 10000
6069 10000
6070 10000
6071 10000
6072 10000
6073 10000
6074 10000
6075 10000
6076 10000
6077 10000
6078 10000
6079 10000
6080 10000
6081 10000
6082 10000
6083 10000
6084 10000
6085 10000
6086 10000
6087 10000
6088 10000
6089 10000
6090 10000
6091 10000
6092 10000
6093 10000
6094 10000
6095 10000
6096 10000
6097 10000
6098 10000
6099 10000
6100 10000
6101 10000
6102 10000
6103 10000
6104 10000
6105 10000
6106 10000
6107 10000
6108 10000
6109 10000
6110 10000
6111 10000
6112 10000
6113 10000
6114 10000
6115 10000
6116 10000
6117 10000
6118 10000
6119 10000
6120 10000
6121 10000
6122 10000
6123 10000
6124 10000
6125 10000
6126 10000
6127 10000
6128 10000
6129 10000
6130 10000
6131 10000
6132 10000
6133 10000
6134 10000
6135 10000
6136 10000
6137 10000
6138 10000
6139 10000
6140 10000
6141 10000
6142 10000
6143 10000
6144 10000
6145 10000
6146 10000
6147 10000
6148 10000
6149 10000
6150 10000
6151 10000
6152 10000

6807 10000
6808 10000
6809 10000
6810 10000
6811 10000
6812 10000
6813 10000
6814 10000
6815 10000
6816 10000
6817 10000
6818 10000
6819 10000
6820 10000
6821 10000
6822 10000
6823 10000
6824 10000
6825 10000
6826 10000
6827 10000
6828 10000
6829 10000
6830 10000
6831 10000
6832 10000
6833 10000
6834 10000
6835 10000
6836 10000
6837 10000
6838 10000
6839 10000
6840 10000
6841 10000
6842 10000
6843 10000
6844 10000
6845 10000
6846 10000
6847 10000
6848 10000
6849 10000
6850 10000
6851 10000
6852 10000
6853 10000
6854 10000
6855 10000
6856 10000
6857 10000
6858 10000
6859 10000
6860 10000
6861 10000
6862 10000
6863 10000
6864 10000
6865 10000
6866 10000
6867 10000
6868 10000
6869 10000
6870 10000
6871 10000
6872 10000
6873 10000
6874 10000
6875 10000
6876 10000
6877 10000
6878 10000
6879 10000
6880 10000
6881 10000
6882 10000
6883 10000
6884 10000
6885 10000
6886 10000
6887 10000
6888 10000
6889 10000
6890 10000
6891 10000
6892 10000
6893 10000
6894 10000
6895 10000
6896 10000
6897 10000

7552 10000
7553 10000
7554 10000
7555 10000
7556 10000
7557 10000
7558 10000
7559 10000
7560 10000
7561 10000
7562 10000
7563 10000
7564 10000
7565 10000
7566 10000
7567 10000
7568 10000
7569 10000
7570 10000
7571 10000
7572 10000
7573 10000
7574 10000
7575 10000
7576 10000
7577 10000
7578 10000
7579 10000
7580 10000
7581 10000
7582 10000
7583 10000
7584 10000
7585 10000
7586 10000
7587 10000
7588 10000
7589 10000
7590 10000
7591 10000
7592 10000
7593 10000
7594 10000
7595 10000
7596 10000
7597 10000
7598 10000
7599 10000
7600 10000
7601 10000
7602 10000
7603 10000
7604 10000
7605 10000
7606 10000
7607 10000
7608 10000
7609 10000
7610 10000
7611 10000
7612 10000
7613 10000
7614 10000
7615 10000
7616 10000
7617 10000
7618 10000
7619 10000
7620 10000
7621 10000
7622 10000
7623 10000
7624 10000
7625 10000
7626 10000
7627 10000
7628 10000
7629 10000
7630 10000
7631 10000
7632 10000
7633 10000
7634 10000
7635 10000
7636 10000
7637 10000
7638 10000
7639 10000
7640 10000
7641 10000
7642 10000

8297 10000
8298 10000
8299 10000
8300 10000
8301 10000
8302 10000
8303 10000
8304 10000
8305 10000
8306 10000
8307 10000
8308 10000
8309 10000
8310 10000
8311 10000
8312 10000
8313 10000
8314 10000
8315 10000
8316 10000
8317 10000
8318 10000
8319 10000
8320 10000
8321 10000
8322 10000
8323 10000
8324 10000
8325 10000
8326 10000
8327 10000
8328 10000
8329 10000
8330 10000
8331 10000
8332 10000
8333 10000
8334 10000
8335 10000
8336 10000
8337 10000
8338 10000
8339 10000
8340 10000
8341 10000
8342 10000
8343 10000
8344 10000
8345 10000
8346 10000
8347 10000
8348 10000
8349 10000
8350 10000
8351 10000
8352 10000
8353 10000
8354 10000
8355 10000
8356 10000
8357 10000
8358 10000
8359 10000
8360 10000
8361 10000
8362 10000
8363 10000
8364 10000
8365 10000
8366 10000
8367 10000
8368 10000
8369 10000
8370 10000
8371 10000
8372 10000
8373 10000
8374 10000
8375 10000
8376 10000
8377 10000
8378 10000
8379 10000
8380 10000
8381 10000
8382 10000
8383 10000
8384 10000
8385 10000
8386 10000
8387 10000

,pickup_datetime,PULocationID,NoOfPickups
0,2001-01-01 00:00,48,1
1,2001-01-01 00:00,151,1
2,2002-02-02 01:00,48,4
3,2002-02-02 01:00,79,1
4,2002-02-02 01:00,87,1


In [13]:
result_all.to_csv('../data/Data_Taxis_2019_Cleaned.csv', index = False, header=True)

In [24]:
# PERSONAL NOTES:
# sort by datetime after compiling unique csv
# interpolate missing values every hour

In [15]:
# 1. Get whole file with 3000 chunks
dfsi = pd.read_csv('./2018_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=10000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))    
    
    # CODE TO RUN
    DistIndex_to_drop = dfs[dfs['trip_distance'] < 0.06].index
    dfs.drop(DistIndex_to_drop, inplace=True)
    
    FareIndex_to_drop = dfs[dfs['fare_amount']<1].index
    dfs.drop(FareIndex_to_drop, inplace=True)

    dfs = dfs[['pickup_datetime','PULocationID']]

    dfs.set_index('pickup_datetime', inplace=True)

    dfs = dfs.to_period("H")

    dfs.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

    dfs['NoOfPickups'] =1
    
    result=dfs.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all2018 = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
result_all2018.head()

0 10000
1 10000
2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 10000
10 10000
11 10000
12 10000
13 10000
14 10000
15 10000
16 10000
17 10000
18 10000
19 10000
20 10000
21 10000
22 10000
23 10000
24 10000
25 10000
26 10000
27 10000
28 10000
29 10000
30 10000
31 10000
32 10000
33 10000
34 10000
35 10000
36 10000
37 10000
38 10000
39 10000
40 10000
41 10000
42 10000
43 10000
44 10000
45 10000
46 10000
47 10000
48 10000
49 10000
50 10000
51 10000
52 10000
53 10000
54 10000
55 10000
56 10000
57 10000
58 10000
59 10000
60 10000
61 10000
62 10000
63 10000
64 10000
65 10000
66 10000
67 10000
68 10000
69 10000
70 10000
71 10000
72 10000
73 10000
74 10000
75 10000
76 10000
77 10000
78 10000
79 10000
80 10000
81 10000
82 10000
83 10000
84 10000
85 10000
86 10000
87 10000
88 10000
89 10000
90 10000
91 10000
92 10000
93 10000
94 10000
95 10000
96 10000
97 10000
98 10000
99 10000
100 10000
101 10000
102 10000
103 10000
104 10000
105 10000
106 10000
107 10000
108 10000
109 10000
110 10000


831 10000
832 10000
833 10000
834 10000
835 10000
836 10000
837 10000
838 10000
839 10000
840 10000
841 10000
842 10000
843 10000
844 10000
845 10000
846 10000
847 10000
848 10000
849 10000
850 10000
851 10000
852 10000
853 10000
854 10000
855 10000
856 10000
857 10000
858 10000
859 10000
860 10000
861 10000
862 10000
863 10000
864 10000
865 10000
866 10000
867 10000
868 10000
869 10000
870 10000
871 10000
872 10000
873 10000
874 10000
875 10000
876 10000
877 10000
878 10000
879 10000
880 10000
881 10000
882 10000
883 10000
884 10000
885 10000
886 10000
887 10000
888 10000
889 10000
890 10000
891 10000
892 10000
893 10000
894 10000
895 10000
896 10000
897 10000
898 10000
899 10000
900 10000
901 10000
902 10000
903 10000
904 10000
905 10000
906 10000
907 10000
908 10000
909 10000
910 10000
911 10000
912 10000
913 10000
914 10000
915 10000
916 10000
917 10000
918 10000
919 10000
920 10000
921 10000
922 10000
923 10000
924 10000
925 10000
926 10000
927 10000
928 10000
929 10000
930 10000


1592 10000
1593 10000
1594 10000
1595 10000
1596 10000
1597 10000
1598 10000
1599 10000
1600 10000
1601 10000
1602 10000
1603 10000
1604 10000
1605 10000
1606 10000
1607 10000
1608 10000
1609 10000
1610 10000
1611 10000
1612 10000
1613 10000
1614 10000
1615 10000
1616 10000
1617 10000
1618 10000
1619 10000
1620 10000
1621 10000
1622 10000
1623 10000
1624 10000
1625 10000
1626 10000
1627 10000
1628 10000
1629 10000
1630 10000
1631 10000
1632 10000
1633 10000
1634 10000
1635 10000
1636 10000
1637 10000
1638 10000
1639 10000
1640 10000
1641 10000
1642 10000
1643 10000
1644 10000
1645 10000
1646 10000
1647 10000
1648 10000
1649 10000
1650 10000
1651 10000
1652 10000
1653 10000
1654 10000
1655 10000
1656 10000
1657 10000
1658 10000
1659 10000
1660 10000
1661 10000
1662 10000
1663 10000
1664 10000
1665 10000
1666 10000
1667 10000
1668 10000
1669 10000
1670 10000
1671 10000
1672 10000
1673 10000
1674 10000
1675 10000
1676 10000
1677 10000
1678 10000
1679 10000
1680 10000
1681 10000
1682 10000

2337 10000
2338 10000
2339 10000
2340 10000
2341 10000
2342 10000
2343 10000
2344 10000
2345 10000
2346 10000
2347 10000
2348 10000
2349 10000
2350 10000
2351 10000
2352 10000
2353 10000
2354 10000
2355 10000
2356 10000
2357 10000
2358 10000
2359 10000
2360 10000
2361 10000
2362 10000
2363 10000
2364 10000
2365 10000
2366 10000
2367 10000
2368 10000
2369 10000
2370 10000
2371 10000
2372 10000
2373 10000
2374 10000
2375 10000
2376 10000
2377 10000
2378 10000
2379 10000
2380 10000
2381 10000
2382 10000
2383 10000
2384 10000
2385 10000
2386 10000
2387 10000
2388 10000
2389 10000
2390 10000
2391 10000
2392 10000
2393 10000
2394 10000
2395 10000
2396 10000
2397 10000
2398 10000
2399 10000
2400 10000
2401 10000
2402 10000
2403 10000
2404 10000
2405 10000
2406 10000
2407 10000
2408 10000
2409 10000
2410 10000
2411 10000
2412 10000
2413 10000
2414 10000
2415 10000
2416 10000
2417 10000
2418 10000
2419 10000
2420 10000
2421 10000
2422 10000
2423 10000
2424 10000
2425 10000
2426 10000
2427 10000

3082 10000
3083 10000
3084 10000
3085 10000
3086 10000
3087 10000
3088 10000
3089 10000
3090 10000
3091 10000
3092 10000
3093 10000
3094 10000
3095 10000
3096 10000
3097 10000
3098 10000
3099 10000
3100 10000
3101 10000
3102 10000
3103 10000
3104 10000
3105 10000
3106 10000
3107 10000
3108 10000
3109 10000
3110 10000
3111 10000
3112 10000
3113 10000
3114 10000
3115 10000
3116 10000
3117 10000
3118 10000
3119 10000
3120 10000
3121 10000
3122 10000
3123 10000
3124 10000
3125 10000
3126 10000
3127 10000
3128 10000
3129 10000
3130 10000
3131 10000
3132 10000
3133 10000
3134 10000
3135 10000
3136 10000
3137 10000
3138 10000
3139 10000
3140 10000
3141 10000
3142 10000
3143 10000
3144 10000
3145 10000
3146 10000
3147 10000
3148 10000
3149 10000
3150 10000
3151 10000
3152 10000
3153 10000
3154 10000
3155 10000
3156 10000
3157 10000
3158 10000
3159 10000
3160 10000
3161 10000
3162 10000
3163 10000
3164 10000
3165 10000
3166 10000
3167 10000
3168 10000
3169 10000
3170 10000
3171 10000
3172 10000

...

9042 10000
9043 10000
9044 10000
9045 10000
9046 10000
9047 10000
9048 10000
9049 10000
9050 10000
9051 10000
9052 10000
9053 10000
9054 10000
9055 10000
9056 10000
9057 10000
9058 10000
9059 10000
9060 10000
9061 10000
9062 10000
9063 10000
9064 10000
9065 10000
9066 10000
9067 10000
9068 10000
9069 10000
9070 10000
9071 10000
9072 10000
9073 10000
9074 10000
9075 10000
9076 10000
9077 10000
9078 10000
9079 10000
9080 10000
9081 10000
9082 10000
9083 10000
9084 10000
9085 10000
9086 10000
9087 10000
9088 10000
9089 10000
9090 10000
9091 10000
9092 10000
9093 10000
9094 10000
9095 10000
9096 10000
9097 10000
9098 10000
9099 10000
9100 10000
9101 10000
9102 10000
9103 10000
9104 10000
9105 10000
9106 10000
9107 10000
9108 10000
9109 10000
9110 10000
9111 10000
9112 10000
9113 10000
9114 10000
9115 10000
9116 10000
9117 10000
9118 10000
9119 10000
9120 10000
9121 10000
9122 10000
9123 10000
9124 10000
9125 10000
9126 10000
9127 10000
9128 10000
9129 10000
9130 10000
9131 10000
9132 10000

9787 10000
9788 10000
9789 10000
9790 10000
9791 10000
9792 10000
9793 10000
9794 10000
9795 10000
9796 10000
9797 10000
9798 10000
9799 10000
9800 10000
9801 10000
9802 10000
9803 10000
9804 10000
9805 10000
9806 10000
9807 10000
9808 10000
9809 10000
9810 10000
9811 10000
9812 10000
9813 10000
9814 10000
9815 10000
9816 10000
9817 10000
9818 10000
9819 10000
9820 10000
9821 10000
9822 10000
9823 10000
9824 10000
9825 10000
9826 10000
9827 10000
9828 10000
9829 10000
9830 10000
9831 10000
9832 10000
9833 10000
9834 10000
9835 10000
9836 10000
9837 10000
9838 10000
9839 10000
9840 10000
9841 10000
9842 10000
9843 10000
9844 10000
9845 10000
9846 10000
9847 10000
9848 10000
9849 10000
9850 10000
9851 10000
9852 10000
9853 10000
9854 10000
9855 10000
9856 10000
9857 10000
9858 10000
9859 10000
9860 10000
9861 10000
9862 10000
9863 10000
9864 10000
9865 10000
9866 10000
9867 10000
9868 10000
9869 10000
9870 10000
9871 10000
9872 10000
9873 10000
9874 10000
9875 10000
9876 10000
9877 10000

10488 10000
10489 10000
10490 10000
10491 10000
10492 10000
10493 10000
10494 10000
10495 10000
10496 10000
10497 10000
10498 10000
10499 10000
10500 10000
10501 10000
10502 10000
10503 10000
10504 10000
10505 10000
10506 10000
10507 10000
10508 10000
10509 10000
10510 10000
10511 10000
10512 10000
10513 10000
10514 10000
10515 10000
10516 10000
10517 10000
10518 10000
10519 10000
10520 10000
10521 10000
10522 10000
10523 10000
10524 10000
10525 10000
10526 10000
10527 10000
10528 10000
10529 10000
10530 10000
10531 10000
10532 10000
10533 10000
10534 10000
10535 10000
10536 10000
10537 10000
10538 10000
10539 10000
10540 10000
10541 10000
10542 10000
10543 10000
10544 10000
10545 10000
10546 10000
10547 10000
10548 10000
10549 10000
10550 10000
10551 10000
10552 10000
10553 10000
10554 10000
10555 10000
10556 10000
10557 10000
10558 10000
10559 10000
10560 10000
10561 10000
10562 10000
10563 10000
10564 10000
10565 10000
10566 10000
10567 10000
10568 10000
10569 10000
10570 10000
1057

11171 10000
11172 10000
11173 10000
11174 10000
11175 10000
11176 10000
11177 10000
11178 10000
11179 10000
11180 10000
11181 10000
11182 10000
11183 10000
11184 10000
11185 10000
11186 10000
11187 10000
11188 10000
11189 10000
11190 10000
11191 10000
11192 10000
11193 10000
11194 10000
11195 10000
11196 10000
11197 10000
11198 10000
11199 10000
11200 10000
11201 10000
11202 10000
11203 10000
11204 10000
11205 10000
11206 10000
11207 10000
11208 10000
11209 10000
11210 10000
11211 10000
11212 10000
11213 10000
11214 10000
11215 10000
11216 10000
11217 10000
11218 10000
11219 10000
11220 10000
11221 10000
11222 10000
11223 4626


,pickup_datetime,PULocationID,NoOfPickups
0,2001-01-01 00:00,43,2
1,2001-01-01 00:00,48,3
2,2001-01-01 00:00,140,2
3,2001-01-01 00:00,141,1
4,2001-01-01 00:00,170,2


In [16]:
result_all2018.to_csv('../data/Data_Taxis_2018_Cleaned.csv', index = False, header=True)

In [17]:
# 1. Get whole file with 3000 chunks
dfsi = pd.read_csv('./2017_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=10000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))    
    
    # CODE TO RUN
    DistIndex_to_drop = dfs[dfs['trip_distance'] < 0.06].index
    dfs.drop(DistIndex_to_drop, inplace=True)
    
    FareIndex_to_drop = dfs[dfs['fare_amount']<1].index
    dfs.drop(FareIndex_to_drop, inplace=True)

    dfs = dfs[['pickup_datetime','PULocationID']]

    dfs.set_index('pickup_datetime', inplace=True)

    dfs = dfs.to_period("H")

    dfs.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

    dfs['NoOfPickups'] =1
    
    result=dfs.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all2017 = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
result_all2017.head()

0 10000
1 10000
2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 10000
10 10000
11 10000
12 10000
13 10000
14 10000
15 10000
16 10000
17 10000
18 10000
19 10000
20 10000
21 10000
22 10000
23 10000
24 10000
25 10000
26 10000
27 10000
28 10000
29 10000
30 10000
31 10000
32 10000
33 10000
34 10000
35 10000
36 10000
37 10000
38 10000
39 10000
40 10000
41 10000
42 10000
43 10000
44 10000
45 10000
46 10000
47 10000
48 10000
49 10000
50 10000
51 10000
52 10000
53 10000
54 10000
55 10000
56 10000
57 10000
58 10000
59 10000
60 10000
61 10000
62 10000
63 10000
64 10000
65 10000
66 10000
67 10000
68 10000
69 10000
70 10000
71 10000
72 10000
73 10000
74 10000
75 10000
76 10000
77 10000
78 10000
79 10000
80 10000
81 10000
82 10000
83 10000
84 10000
85 10000
86 10000
87 10000
88 10000
89 10000
90 10000
91 10000
92 10000
93 10000
94 10000
95 10000
96 10000
97 10000
98 10000
99 10000
100 10000
101 10000
102 10000
103 10000
104 10000
105 10000
106 10000
107 10000
108 10000
109 10000
110 10000


831 10000
832 10000
833 10000
834 10000
835 10000
836 10000
837 10000
838 10000
839 10000
840 10000
841 10000
842 10000
843 10000
844 10000
845 10000
846 10000
847 10000
848 10000
849 10000
850 10000
851 10000
852 10000
853 10000
854 10000
855 10000
856 10000
857 10000
858 10000
859 10000
860 10000
861 10000
862 10000
863 10000
864 10000
865 10000
866 10000
867 10000
868 10000
869 10000
870 10000
871 10000
872 10000
873 10000
874 10000
875 10000
876 10000
877 10000
878 10000
879 10000
880 10000
881 10000
882 10000
883 10000
884 10000
885 10000
886 10000
887 10000
888 10000
889 10000
890 10000
891 10000
892 10000
893 10000
894 10000
895 10000
896 10000
897 10000
898 10000
899 10000
900 10000
901 10000
902 10000
903 10000
904 10000
905 10000
906 10000
907 10000
908 10000
909 10000
910 10000
911 10000
912 10000
913 10000
914 10000
915 10000
916 10000
917 10000
918 10000
919 10000
920 10000
921 10000
922 10000
923 10000
924 10000
925 10000
926 10000
927 10000
928 10000
929 10000
930 10000


1592 10000
1593 10000
1594 10000
1595 10000
1596 10000
1597 10000
1598 10000
1599 10000
1600 10000
1601 10000
1602 10000
1603 10000
1604 10000
1605 10000
1606 10000
1607 10000
1608 10000
1609 10000
1610 10000
1611 10000
1612 10000
1613 10000
1614 10000
1615 10000
1616 10000
1617 10000
1618 10000
1619 10000
1620 10000
1621 10000
1622 10000
1623 10000
1624 10000
1625 10000
1626 10000
1627 10000
1628 10000
1629 10000
1630 10000
1631 10000
1632 10000
1633 10000
1634 10000
1635 10000
1636 10000
1637 10000
1638 10000
1639 10000
1640 10000
1641 10000
1642 10000
1643 10000
1644 10000
1645 10000
1646 10000
1647 10000
1648 10000
1649 10000
1650 10000
1651 10000
1652 10000
1653 10000
1654 10000
1655 10000
1656 10000
1657 10000
1658 10000
1659 10000
1660 10000
1661 10000
1662 10000
1663 10000
1664 10000
1665 10000
1666 10000
1667 10000
1668 10000
1669 10000
1670 10000
1671 10000
1672 10000
1673 10000
1674 10000
1675 10000
1676 10000
1677 10000
1678 10000
1679 10000
1680 10000
1681 10000
1682 10000

2337 10000
2338 10000
2339 10000
2340 10000
2341 10000
2342 10000
2343 10000
2344 10000
2345 10000
2346 10000
2347 10000
2348 10000
2349 10000
2350 10000
2351 10000
2352 10000
2353 10000
2354 10000
2355 10000
2356 10000
2357 10000
2358 10000
2359 10000
2360 10000
2361 10000
2362 10000
2363 10000
2364 10000
2365 10000
2366 10000
2367 10000
2368 10000
2369 10000
2370 10000
2371 10000
2372 10000
2373 10000
2374 10000
2375 10000
2376 10000
2377 10000
2378 10000
2379 10000
2380 10000
2381 10000
2382 10000
2383 10000
2384 10000
2385 10000
2386 10000
2387 10000
2388 10000
2389 10000
2390 10000
2391 10000
2392 10000
2393 10000
2394 10000
2395 10000
2396 10000
2397 10000
2398 10000
2399 10000
2400 10000
2401 10000
2402 10000
2403 10000
2404 10000
2405 10000
2406 10000
2407 10000
2408 10000
2409 10000
2410 10000
2411 10000
2412 10000
2413 10000
2414 10000
2415 10000
2416 10000
2417 10000
2418 10000
2419 10000
2420 10000
2421 10000
2422 10000
2423 10000
2424 10000
2425 10000
2426 10000
2427 10000

3082 10000
3083 10000
3084 10000
3085 10000
3086 10000
3087 10000
3088 10000
3089 10000
3090 10000
3091 10000
3092 10000
3093 10000
3094 10000
3095 10000
3096 10000
3097 10000
3098 10000
3099 10000
3100 10000
3101 10000
3102 10000
3103 10000
3104 10000
3105 10000
3106 10000
3107 10000
3108 10000
3109 10000
3110 10000
3111 10000
3112 10000
3113 10000
3114 10000
3115 10000
3116 10000
3117 10000
3118 10000
3119 10000
3120 10000
3121 10000
3122 10000
3123 10000
3124 10000
3125 10000
3126 10000
3127 10000
3128 10000
3129 10000
3130 10000
3131 10000
3132 10000
3133 10000
3134 10000
3135 10000
3136 10000
3137 10000
3138 10000
3139 10000
3140 10000
3141 10000
3142 10000
3143 10000
3144 10000
3145 10000
3146 10000
3147 10000
3148 10000
3149 10000
3150 10000
3151 10000
3152 10000
3153 10000
3154 10000
3155 10000
3156 10000
3157 10000
3158 10000
3159 10000
3160 10000
3161 10000
3162 10000
3163 10000
3164 10000
3165 10000
3166 10000
3167 10000
3168 10000
3169 10000
3170 10000
3171 10000
3172 10000

...

9042 10000
9043 10000
9044 10000
9045 10000
9046 10000
9047 10000
9048 10000
9049 10000
9050 10000
9051 10000
9052 10000
9053 10000
9054 10000
9055 10000
9056 10000
9057 10000
9058 10000
9059 10000
9060 10000
9061 10000
9062 10000
9063 10000
9064 10000
9065 10000
9066 10000
9067 10000
9068 10000
9069 10000
9070 10000
9071 10000
9072 10000
9073 10000
9074 10000
9075 10000
9076 10000
9077 10000
9078 10000
9079 10000
9080 10000
9081 10000
9082 10000
9083 10000
9084 10000
9085 10000
9086 10000
9087 10000
9088 10000
9089 10000
9090 10000
9091 10000
9092 10000
9093 10000
9094 10000
9095 10000
9096 10000
9097 10000
9098 10000
9099 10000
9100 10000
9101 10000
9102 10000
9103 10000
9104 10000
9105 10000
9106 10000
9107 10000
9108 10000
9109 10000
9110 10000
9111 10000
9112 10000
9113 10000
9114 10000
9115 10000
9116 10000
9117 10000
9118 10000
9119 10000
9120 10000
9121 10000
9122 10000
9123 10000
9124 10000
9125 10000
9126 10000
9127 10000
9128 10000
9129 10000
9130 10000
9131 10000
9132 10000

9787 10000
9788 10000
9789 10000
9790 10000
9791 10000
9792 10000
9793 10000
9794 10000
9795 10000
9796 10000
9797 10000
9798 10000
9799 10000
9800 10000
9801 10000
9802 10000
9803 10000
9804 10000
9805 10000
9806 10000
9807 10000
9808 10000
9809 10000
9810 10000
9811 10000
9812 10000
9813 10000
9814 10000
9815 10000
9816 10000
9817 10000
9818 10000
9819 10000
9820 10000
9821 10000
9822 10000
9823 10000
9824 10000
9825 10000
9826 10000
9827 10000
9828 10000
9829 10000
9830 10000
9831 10000
9832 10000
9833 10000
9834 10000
9835 10000
9836 10000
9837 10000
9838 10000
9839 10000
9840 10000
9841 10000
9842 10000
9843 10000
9844 10000
9845 10000
9846 10000
9847 10000
9848 10000
9849 10000
9850 10000
9851 10000
9852 10000
9853 10000
9854 10000
9855 10000
9856 10000
9857 10000
9858 10000
9859 10000
9860 10000
9861 10000
9862 10000
9863 10000
9864 10000
9865 10000
9866 10000
9867 10000
9868 10000
9869 10000
9870 10000
9871 10000
9872 10000
9873 10000
9874 10000
9875 10000
9876 10000
9877 10000

10488 10000
10489 10000
10490 10000
10491 10000
10492 10000
10493 10000
10494 10000
10495 10000
10496 10000
10497 10000
10498 10000
10499 10000
10500 10000
10501 10000
10502 10000
10503 10000
10504 10000
10505 10000
10506 10000
10507 10000
10508 10000
10509 10000
10510 10000
10511 10000
10512 10000
10513 10000
10514 10000
10515 10000
10516 10000
10517 10000
10518 10000
10519 10000
10520 10000
10521 10000
10522 10000
10523 10000
10524 10000
10525 10000
10526 10000
10527 10000
10528 10000
10529 10000
10530 10000
10531 10000
10532 10000
10533 10000
10534 10000
10535 10000
10536 10000
10537 10000
10538 10000
10539 10000
10540 10000
10541 10000
10542 10000
10543 10000
10544 10000
10545 10000
10546 10000
10547 10000
10548 10000
10549 10000
10550 10000
10551 10000
10552 10000
10553 10000
10554 10000
10555 10000
10556 10000
10557 10000
10558 10000
10559 10000
10560 10000
10561 10000
10562 10000
10563 10000
10564 10000
10565 10000
10566 10000
10567 10000
10568 10000
10569 10000
10570 10000
1057

11171 10000
11172 10000
11173 10000
11174 10000
11175 10000
11176 10000
11177 10000
11178 10000
11179 10000
11180 10000
11181 10000
11182 10000
11183 10000
11184 10000
11185 10000
11186 10000
11187 10000
11188 10000
11189 10000
11190 10000
11191 10000
11192 10000
11193 10000
11194 10000
11195 10000
11196 10000
11197 10000
11198 10000
11199 10000
11200 10000
11201 10000
11202 10000
11203 10000
11204 10000
11205 10000
11206 10000
11207 10000
11208 10000
11209 10000
11210 10000
11211 10000
11212 10000
11213 10000
11214 10000
11215 10000
11216 10000
11217 10000
11218 10000
11219 10000
11220 10000
11221 10000
11222 10000
11223 10000
11224 10000
11225 10000
11226 10000
11227 10000
11228 10000
11229 10000
11230 10000
11231 10000
11232 10000
11233 10000
11234 10000
11235 10000
11236 10000
11237 10000
11238 10000
11239 10000
11240 10000
11241 10000
11242 10000
11243 10000
11244 10000
11245 10000
11246 10000
11247 10000
11248 10000
11249 10000
11250 10000
11251 10000
11252 10000
11253 10000
1125

,pickup_datetime,PULocationID,NoOfPickups
0,2001-01-01 23:00,230,1
1,2002-12-31 23:00,230,1
2,2003-01-01 00:00,163,1
3,2003-01-01 00:00,263,1
4,2008-12-31 09:00,161,1


In [18]:
result_all2017.to_csv('../data/Data_Taxis_2017_Cleaned.csv', index = False, header=True)

In [ ]:
# Añadir filtro por año porque hay entradas de otros años, por arriba y por abajo!